In [45]:
import pandas as pd
import pprint
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from nltk.corpus import stopwords 
import nltk
import csv
import numpy as np
import re
import emoji
from keras.models import Sequential
from keras.layers import Dense, Dropout
from nltk.stem.porter import PorterStemmer
from string import digits
from autocorrect import spell
import preprocessor as p
import requests
import json
import wordsegment as ws

In [46]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
stemmer = PorterStemmer()
ws.load()
p.set_options(p.OPT.HASHTAG, p.OPT.NUMBER, p.OPT.EMOJI, p.OPT.SMILEY)
def format_tweet(row):
    
    p.set_options(p.OPT.HASHTAG, p.OPT.NUMBER, p.OPT.EMOJI, p.OPT.SMILEY,p.OPT.URL, p.OPT.MENTION )
    parsed_tweet = p.parse(row)
    extra = []
    if parsed_tweet.urls:
        for each in parsed_tweet.urls:
            row = row[:each.start_index]+ ' URL '+row[each.end_index:]
            
    if parsed_tweet.mentions:
        for each in parsed_tweet.mentions:
            row = row[:each.start_index]+ ' MNTN '+row[each.end_index:]
            
    if parsed_tweet.hashtags:
        for each in parsed_tweet.hashtags:
            extra+=ws.segment(each.match)
    #print(row)
            
    if parsed_tweet.numbers:
        for each in parsed_tweet.numbers:
            extra.append('NMBR')
    
    if parsed_tweet.hashtags:
        for each in parsed_tweet.hashtags:
            extra.append('HASH')
            
    if parsed_tweet.emojis:
        for each in parsed_tweet.emojis:
            extra.append('EMOJI')
    
    if parsed_tweet.smileys:
        for each in parsed_tweet.smileys:
            extra.append('SMIL')
    
    p.set_options(p.OPT.URL,p.OPT.MENTION,p.OPT.NUMBER)
    row = p.clean(row)
    tokens = tknzr.tokenize(row)
    tokens_formatted = []
    
    for i in range(len(tokens)):
           if tokens[i] in emoji.UNICODE_EMOJI:
                tokens[i] = emoji.demojize(tokens[i])
    
    for each in tokens:
        if each not in stop_words and len(each)> 0:
            tokens_formatted.append(each)
            
    tokens_formatted+=extra
    return " ".join(tokens_formatted).lower()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tharindu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def format_data(row_data):
    data = []
    i=0
    for row in row_data:
        
        i+=1
        data.append(format_tweet(row))
    return data

In [47]:
data = []
classes = []
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
with open('/Users/Tharindu/Pictures/ML/all/train.csv', 'r', encoding="utf8") as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        try:
            classes.append(int(row[1]))
        except:
            continue
        data.append(row[2])
traing_data = format_data(data)
i=0
for cl,d in zip(classes,traing_data):
    i+=1
    print(i," ",cl," : ",d)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tharindu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
1   1  :  sweet united nations video . just time christmas . #imagine #noreligion #irony url imagine no religion irony hash hash hash
2   1  :  mntn we rumored talked erv's agent ... angels asked ed escobar ... that's hardly nothing #sarcasm ;) sarcasm hash smil
3   1  :  hey ! nice see minnesota / nd winter weather #not not hash
4   0  :  episodes left i'm dying nmbr
5   1  :  i can't breathe ! chosen notable quote year annual list released yale university librarian #irony irony hash
6   0  :  you're never old footie pajamas . url
7   1  :  nothing makes happier getting highway seeing break lights light like christmas tree .. #not not hash
8   0  :  : 30 opening first beer gonna long night / day nmbr
9   0  :  mntn think would support guy knocked daughter ? rice deserve support .
10   0  :  mntn you allowed open christmas da

197   0  :  #qz8501 asks permission #takeoff : #towercontrol #surubaya : yeah .. go ... away . qz8501 take off tower control suruba ya hash hash hash hash
198   0  :  mntn mntn wer - more artists | #winsunburnpasses win sunburn passes nmbr nmbr hash
199   1  :  " mntn mntn aying provinces like
200   0  :  church sign day : how would feel celebrated birthday without ? :speak-no-evil_monkey: #stoleem #thereasonfortheseason stole em the reason for the season hash hash emoji
201   1  :  mntn : i mntn he word man in it ! ! just sexist saying mail man instead mail carrier . #sarcasm #michiganman sarcasm michiganman hash hash
202   1  :  did know ? if reading right blind ... lol . #sarcasm sarcasm hash
203   0  :  mntn mntn wont talk bodies cops killed bk ambulanced immediately . elitism finest . nmbr
204   1  :  love weather #not not hash
205   0  :  mntn pssst . you spelt ' thing ' wrong | | #irony irony hash
206   0  :  url what day sabbath ? #follow #jesus #christ #fbc #fbcjax url follow 

397   1  :  marvin lewis clearly thinks highly johnny football #not #norespect mntn not no respect hash hash
398   1  :  finals week draining body much ruining life places . lovely ... #not not hash
399   1  :  about write math final really excited #not #somethingrandom url not something random hash hash
400   0  :  pulled school first time life good mood find doesnt start another hours #not #blessed not blessed nmbr hash hash
401   1  :  " dragging feet
402   0  :  i say misty johnny mathis one favourites url
403   1  :  mntn mntn ly scary bollywood film i've watched bhootnaath #sarcasm sarcasm hash
404   0  :  read url news ' #middle #east #updates # / #assad : #us-led #airstrikes #not #serious #or #efficient ' middle east updates assad us air strikes not serious or efficient hash hash hash hash hash hash hash hash hash hash hash
405   1  :  love working hard trying fix people's fuck ups #sarcasm sarcasm hash
406   1  :  went neck deep swamp today fun #not #chilly not chilly hash has

597   0  :  and christmas shopping done .
598   1  :  mntn lol cthulhu ? ? funny autocorrect helpful #not not hash
599   0  :  my son thoughtful loves taking snacks class . # always thinking others #raising right #not selfish raising not hash hash hash
600   1  :  a year old ( whatever age idgaf ) acting like immature little mntn # $ % & public ? yup classy one . #sarcasm #sonotclassy sarcasm so not classy nmbr hash hash hash
601   0  :  " update ! punt ostrich ? url january nmbr
602   1  :  hiding plain sight . by #michaelbeerens ( url ) #graffiti #streetart #irony url michael beer ens graffiti street art irony hash hash hash hash
603   0  :  " mntn mntn en get cynical
604   1  :  today awesome #not not hash
605   0  :  " mntn nm
606   1  :  " heh
607   1  :  mntn : love randomly waking : 43 #not everyday multiple times :unamused_face: :unamused_face: not nmbr hash emoji emoji
608   1  :  christmas shopping hrs sleep going fun ! ! ! #not not nmbr hash
609   0  :  stay tuned drunk vide

878   0  :  " mntn mntn look bridge without crying . " " "
879   1  :  love i never fall asleep #not #insomnia not insomnia hash hash
880   0  :  mntn ? waste time ... :) smil
881   0  :  " can trade mccain ? biden helps republicans mccain . in time
882   1  :  happy monday ! #not hate mondays . not hash
883   0  :  is holding rule anymore ?
884   0  :  mntn doubt . and wonder xmas makes ppl crazy ? !
885   0  :  " mntn : mntn yo lazy butt outta bed
886   0  :  " much lies deceit
887   0  :  " i liked mntn video mntn url wwe k15 ( ps4 ) my career mode ep : #12 : " " triple threat " " | wwe k15 " 12 nmbr nmbr hash
888   1  :  " hey
889   0  :  this time change crazy . everyones like woohoo lets live life ! & i'm like & i slept yet . nmbr nmbr
890   1  :  why playoff legend romo ? mntn #sarcasm sarcasm hash
891   0  :  #palestinian #authority - - #abbas #trades #stalemate #confrontation #icc #move . url via mntn palestinian authority abbas trades stalemate confrontation icc move hash has

...

2796   1  :  and round ligament pain returns . yay #not #pregnancyproblems not pregnancy problems hash hash
2797   1  :  " mntn great job reporting mentally unstable man calling " " drug crazed " " #sarcasm " sarcasm hash
2798   1  :  mntn - russell brand discussed today much london mansion worth .. marx married baroness #irony #scum #weak irony scum weak hash hash hash
2799   1  :  " mntn hey
2800   1  :  " mntn logic invention kuffar
2801   1  :  " just hunch
2802   0  :  " mntn well since he's basically employee oil & gas sector
2803   0  :  just hangin ' dawg #eyesup url eyes up hash
2804   1  :  sitting around listening sad music really helps depression . #not not hash
2805   1  :  still awake sooo that's nice ... #not not hash
2806   1  :  " watching move ' begin again ' verisimilitude overpowering
2807   1  :  i love mom #sarcasm sarcasm hash
2808   0  :  some people listen .. ah well guess they'll learn hard way
2809   0  :  imagine high school today delusional dad making fool 

2996   0  :  ã ¢ ëœå find hip hop mixtape promotion packages url
2997   1  :  well throwing : 00 always fun :smiling_face: | #not not nmbr hash emoji
2998   1  :  / 2 hour nap ct scan hospital . perfect timing . #not not nmbr nmbr hash
2999   1  :  it's nice see continuity gender struggles . #not not hash
3000   0  :  sad hear mntn passing . his family friends prayers today . #rip mntn rip hash
3001   0  :  when i'm hw time watching series . #1am #now #still #not #finish #my #work ! # url 1am now still not finish my work hash hash hash hash hash hash hash hash
3002   1  :  the routine texting irks nerves i guess thought count #not #needanewroutine not need a new routine hash hash
3003   0  :  mntn i know severity charged . i read internet . praying & wife .
3004   1  :  i wonderful gf ! #sarcasm sarcasm hash
3005   1  :  who ever thought moving middle december would peaceful ? #sarcasm sarcasm hash
3006   0  :  retweet mntn book signing & screening area . url url
3007   0  :  oh ffs bo

3296   1  :  #muslimapologies – tweets explain what muslims should sorry ! url #sarcasm muslim apologies sarcasm nmbr hash hash smil
3297   0  :  " mntn yeah i
3298   0  :  mntn at : 30am ? who's gonna favourite ? an owl ? nmbr
3299   0  :  wasn't playoff supposed make process easier ? ? #ncaa ncaa hash
3300   0  :  " same whoopi said rape yo roman polanski . not " " rape rape " " b / c drugged . url via " mntn nmbr
3301   1  :  mntn those failed develop hair bald hear talking abr development #irony irony hash
3302   1  :  so glad work mornings #not not hash
3303   1  :  " preparing sample answers rehearsal exam qs digital media google drive
3304   0  :  " casual meal mate
3305   1  :  " lucky i've finished coursework
3306   1  :  hahahhaha really made jelouse #not not hash
3307   1  :  " mntn mntn nabb i ' mntn decide cover says " " lonely widower ' . #not " not hash
3308   1  :  another day lovely comcast subscriber . #not #neverworks mntn mntn ity mntn not never works hash hash
3309

3496   1  :  well .. nice text wake #not . not hash
3497   1  :  " love adverts woman black say " " sleep weeks " " ... way sell :flushed_face: #not #sheneverleft " not she never left nmbr hash hash emoji
3498   1  :  " spiky popularity name " " spike " " u . s . rt mntn baby name #irony . #dataviz source : url " irony dataviz hash hash
3499   1  :  jfk baggage idiots accused of stealing shit right outa your luggage ! | shocked ? | ummm so #not | they hire morons ! | put the army in there ! not hash
3500   1  :  " mntn wait ... there's " " new wii u " " ? #sarcasm " sarcasm hash
3501   1  :  perfect way start morning #not not hash
3502   0  :  getting hit tsa agent thinks first name bowie #wifemeup #not wife me up not hash hash
3503   0  :  mntn i'm hurt i still love
3504   1  :  on small diet #not url not hash
3505   1  :  mntn we study hard .. :books: :speak-no-evil_monkey: #not #procrastinators :people_with_bunny_ears: not procrastinators hash hash emoji emoji emoji smil smil
3506  

3696   0  :  incredible . rt mntn : the red sox agreement justin masterson .
3697   1  :  mntn mntn ? ? ? no verbal submission ? #sarcasm sarcasm hash
3698   1  :  finals week plus hearing grandma icu = amazing #sarcasm #letmegohome sarcasm let me go home hash hash
3699   1  :  well said mntn ! i one fed women breastfeeding ostentatiously ! #sarcasm sarcasm hash
3700   0  :  mntn mntn t's like maldives #seaandwhitesands sea and whitesands hash
3701   1  :  " good choice stay late
3702   1  :  mntn mntn enneee mntn . you see videos photos taken two hours shooting . #not not hash
3703   1  :  mntn i boyfriend ! ( #not ) * hair flip walks away * not hash
3704   1  :  i love half group really committed half give crap ! #perfection #sarcasm perfection sarcasm hash hash
3705   1  :  mntn mntn abour mntn somewhere can't quite put finger #sarcasm sarcasm hash
3706   0  :  body language confidence - easiest thing make difference : url via > mntn
3707   0  :  mntn loughton .. and basildon u ?
37

In [48]:
traing_data

['sweet united nations video . just time christmas . #imagine #noreligion #irony url imagine no religion irony hash hash hash',
 "mntn we rumored talked erv's agent ... angels asked ed escobar ... that's hardly nothing #sarcasm ;) sarcasm hash smil",
 'hey ! nice see minnesota / nd winter weather #not not hash',
 "episodes left i'm dying nmbr",
 "i can't breathe ! chosen notable quote year annual list released yale university librarian #irony irony hash",
 "you're never old footie pajamas . url",
 'nothing makes happier getting highway seeing break lights light like christmas tree .. #not not hash',
 ': 30 opening first beer gonna long night / day nmbr',
 'mntn think would support guy knocked daughter ? rice deserve support .',
 'mntn you allowed open christmas day !',
 '" oh',
 '" but instead',
 'mntn :pouting_face: bloody i upstairs getting changed ! emoji',
 "cold warmth suffuse one's cheeks pink ( colour / tone ) ... do understand underlying difference & texture ?",
 'just great mo

In [49]:
submission_test_set_row = []
with open('/Users/Tharindu/Pictures/ML/all/test.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        try:
            row = row[1]
        except Exceptions as ex:
            print(ex)
            continue
        submission_test_set_row.append(row)
submission_test_set_formatted = format_data(submission_test_set_row)

In [50]:
submission_test_set_formatted


['text',
 'mntn can u help ? | | more conservatives needed #tsu + get paid posting stuff like ! | | you $ go url tsu nmbr hash',
 '" just walked #starbucks asked " " tall blonde " " hahahaha #irony " starbucks irony hash hash',
 '#not gonna win url not hash',
 'mntn he exactly sort person . weirdo !',
 "so much #sarcasm work mate / 10 #boring % #dead mate full #shit absolutely #sleeping mate can't handle #sarcasm sarcasm boring dead shit sleeping sarcasm nmbr nmbr hash hash hash hash hash hash",
 'corny jokes absolute favorite',
 '" people complain backround pic i feel like " " hey blame',
 '" mntn mntn n',
 '" christian expects fifa sleep bed tonight smil',
 '" people tell people anxiety " " stop worrying " " favorite kind people #not #educateyourself " not educate yourself hash hash',
 "most important thing i've learned school url",
 '#orionlaunch underway - yeah ! #orion url orion launch orion hash hash',
 'i love context large ensemble fridays ! ! ! der favourite #sarcasm #getthefu

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1,3
                                         ), min_df=2)

X_train_counts = count_vect.fit_transform(traing_data)
X_train_counts.shape


(3834, 5834)

In [52]:
import sklearn.feature_extraction.text as T

tfidf_transformer = T.TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [53]:
from sklearn.decomposition import PCA
pca = PCA(n_components=32)
result_pca = pca.fit_transform(X_train_tfidf.toarray())
print(result_pca)
print(pca)

[[-0.14670876  0.03425336  0.13922154 ...  0.04198772 -0.01927416
  -0.0614348 ]
 [ 0.06946592  0.23362306 -0.07294429 ... -0.01690853 -0.04195587
  -0.01788037]
 [-0.06620749 -0.1820828  -0.14453863 ... -0.06436423 -0.01518083
  -0.03808118]
 ...
 [ 0.01704148  0.00656708 -0.08694125 ... -0.0655221   0.02948086
   0.10750236]
 [-0.35861127  0.00979729  0.33845108 ... -0.00573624 -0.00571545
   0.00910048]
 [-0.08642    -0.05719998 -0.03236559 ... -0.01888479  0.00714438
  -0.01786362]]
PCA(copy=True, iterated_power='auto', n_components=32, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)


In [54]:
traing_transformed = X_train_tfidf

In [55]:
record_accuracy = True

In [56]:
fraction = 3400
svd_transformed =traing_transformed
X_test = svd_transformed[fraction:]
Y_test = classes[fraction:]
X_train = svd_transformed[:fraction]
y_train = classes[:fraction]

In [59]:
from sklearn.svm import SVC
clf_svc = SVC(C=0.3,gamma='auto',kernel="linear",random_state=10,probability=True)
clf_svc.fit(X_train, y_train)
print(clf_svc.score(X_train, y_train))
t2 = clf_svc.score(X_test,Y_test)
print(t2)
if record_accuracy:
    v2=t2

0.8608823529411764
0.8341013824884793


In [61]:
from sklearn.ensemble import RandomForestClassifier
clf_rdf = RandomForestClassifier(n_estimators=500,max_depth=6,random_state=40)

clf_rdf.fit(X_train, y_train)
print(clf_rdf.score(X_train, y_train))
t4=clf_rdf.score(X_test,Y_test)
print(t4)
if record_accuracy:
    v4=t4

0.8311764705882353
0.8110599078341014


In [63]:
from sklearn.naive_bayes import MultinomialNB
clf_nb = MultinomialNB()
alpha=[]
for i in range(1,1000):
    alpha.append(i/1.0)
parameters = {'alpha':alpha}
search = GridSearchCV(clf_nb, parameters, cv=5)
clf_nb = search.fit(X_train, y_train)


print(clf_nb.score(X_train, y_train))
t6=clf_nb.score(X_test,Y_test)
print(t6)
if record_accuracy:
    v6=t6
search.best_params_

0.8355882352941176
0.8271889400921659


{'alpha': 467.0}

In [35]:
X_submission_counts = count_vect.transform(submission_test_set_formatted)
submission_transormed=tfidf_transformer.transform(X_submission_counts)
print(submission_transormed.shape)

predicted_clf_svc = clf_svc.predict(submission_transormed)
predicted_clf_rdf = clf_rdf.predict(submission_transormed)
predicted_clf_nb = clf_nb.predict(submission_transormed)
rs_sub = []
for p1,p2,p3 in zip(predicted_clf_svc,predicted_clf_rdf,predicted_clf_nb):
    e = round((p1+p2+p3)/3)
    rs_sub.append(int(e))
predicted=rs_sub

(785, 5834)


In [36]:
predicted

[0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,


In [44]:
import csv
with open('result.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Index','Label'])
    for i in range(len(predicted)):
        writer.writerow([i+1,predicted[i]])